In [1]:
import os
import cv2
import random
import numpy as np
import pandas as pd
import seaborn as sns
import tensorflow as tf
from tensorflow import keras
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
from keras.models import Sequential
from tensorflow.keras import regularizers as reg
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report
from keras.layers import Dense, Conv2D, MaxPooling2D, ZeroPadding2D, Flatten, GlobalAveragePooling2D, InputLayer, Dropout, SpatialDropout2D, BatchNormalization, Resizing, Rescaling, RandomFlip, RandomRotation

In [2]:
# Image Configuration 
batch_size = 32

# Images Dirs 
train_dir = 'data/train'
test_dir = 'data/test'

In [3]:
## image generator 
train_data_gen = ImageDataGenerator(rescale=1./255)
validation_data_gen = ImageDataGenerator(rescale=1./255)

In [4]:
# Preprocess all test images
train_generator = train_data_gen.flow_from_directory(
        train_dir,
        target_size=(200, 200),
        batch_size=64,
        color_mode="grayscale",
        class_mode='categorical')

# Preprocess all train images
test_generator = validation_data_gen.flow_from_directory(
        test_dir,
        target_size=(200, 200),
        batch_size=64,
        color_mode="grayscale",
        class_mode='categorical')

Found 274 images belonging to 5 classes.
Found 180 images belonging to 5 classes.


In [5]:
class_names = list(train_generator.class_indices.keys())
print(class_names)

['chris_evans', 'chris_hemsworth', 'mark_ruffalo', 'robert_downey_jr', 'scarlett_johansson']


In [6]:
# create model structure
model = Sequential()

model.add(Conv2D(32, kernel_size=(3, 3), activation='relu', input_shape=(200, 200, 1)))
model.add(Conv2D(64, kernel_size=(3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))

model.add(Conv2D(128, kernel_size=(3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Conv2D(128, kernel_size=(3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))

model.add(Flatten())
model.add(Dense(1024, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(5, activation='softmax'))

cv2.ocl.setUseOpenCL(False)

model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

# Train the neural network/model
model_info = model.fit_generator(
        train_generator,
        epochs=20,
        validation_data=test_generator,
)

/var/folders/21/f04jhrl502sbf8zzw0k0kgdw0000gn/T/ipykernel_2181/38885059.py:25: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  model_info = model.fit_generator(


Epoch 1/20
5/5 [==============================] - 4s 625ms/step - loss: 4.3183 - accuracy: 0.2153 - val_loss: 1.6053 - val_accuracy: 0.3611
Epoch 2/20
5/5 [==============================] - 2s 420ms/step - loss: 1.6094 - accuracy: 0.2336 - val_loss: 1.5570 - val_accuracy: 0.4722
Epoch 3/20
5/5 [==============================] - 2s 423ms/step - loss: 1.5926 - accuracy: 0.2409 - val_loss: 1.5035 - val_accuracy: 0.3667
Epoch 4/20
5/5 [==============================] - 2s 418ms/step - loss: 1.6009 - accuracy: 0.3029 - val_loss: 1.5622 - val_accuracy: 0.5667
Epoch 5/20
5/5 [==============================] - 2s 402ms/step - loss: 1.5738 - accuracy: 0.3066 - val_loss: 1.5244 - val_accuracy: 0.5389
Epoch 6/20
5/5 [==============================] - 2s 405ms/step - loss: 1.4714 - accuracy: 0.4124 - val_loss: 1.3670 - val_accuracy: 0.6778
Epoch 7/20
5/5 [==============================] - 2s 405ms/step - loss: 1.3320 - accuracy: 0.5401 - val_loss: 0.8830 - val_accuracy: 0.7056
Epoch 8/20
5/5 [====

In [ ]:
# save model structure in jason file
model_json = model.to_json()
with open("cnn_model.json", "w") as json_file:
    json_file.write(model_json)

# save trained model weight in .h5 file
model.save_weights('cnn_model.h5')